# Introduction

Après avoir parcouru le fichier, nous allons commencer à le modifier

# Import des modules

In [ ]:
import pandas as pd

# Lecture des données

In [ ]:
df = pd.read_csv("datas.csv", index_col=0)

# Quelques rappels

### 10 première lignes

In [ ]:
df.head(10)

### 10 dernière lignes

In [ ]:
df.tail(10)

### Liste des colonnes

In [ ]:
df.columns

### Dimensions du tableau

In [ ]:
df.shape

### Première infos statistiques

In [ ]:
df.describe()

### Décompte des valeurs présentes

In [ ]:
df.count()

# Gestion des manquants

### On peut par exemple remplacer les valeurs manquante par un objet de notre choix

In [ ]:
df.fillna(0)

In [ ]:
df.fillna('toto')

### On peut remplacer les valeurs manquantes d'une colonne spécifique

In [ ]:
df.CancellationCode.fillna('Z')

### En l'état, on n'a pas modifié le DataFrame, on a juste affiché ce qu'est le DataFrame en remplaçant les valeurs manquantes.
### Pour réellement modifier le DataFrame, on utilise l'argument inplace=True

In [ ]:
df.fillna(0, inplace=True)

In [ ]:
df.head()

# Suppression de colonnes
Les colonnes Year et Month ne servent à rien puisque le fichier ne concerne que mai 2006

In [ ]:
df.drop(['Year', 'Month'], axis=1, inplace=True)

In [ ]:
df.shape

In [ ]:
df.head()

### Exercice : supprimer les colonnes TaxiIn et TaxiOut

# Suppression de lignes
Les vols annulés ne nous intéressent pas

In [ ]:
df = df[df.Cancelled == 0]

In [ ]:
df.shape

### Exercice : faire la même chose avec les vols dévies

### Les colonnes Diverted, Cancelled et CancellationCode sont devenues inutiles du fait de l'extraction précédente

### Pour finir, d'autres colonnes ne nous intéresseront pas : FlightNum, TailNum

# Tri des lignes

### On souhaite trier les lignes selon l'heure de départ

In [ ]:
df.sort_values(by='DepTime', inplace=True)

In [ ]:
df.head()

### Exercice : trouver comment trier à la fois par jour du mois et heure de départ

# Création de colonnes
On souhaite créer une colonne renseignant, à partir des différents types de retard, le retard total de chaque vol

### Création de la colonne du retard total (il existe plusieurs méthodes)

In [ ]:
df['TotalDelay'] = df.CarrierDelay + df.WeatherDelay + df.NASDelay + df.SecurityDelay + df.LateAircraftDelay

In [ ]:
df.head()

### Exercice : Chercher comment créer la colonne caractérisant si l'avion est en retard ou non (il existe plusieurs méthodes)

# Supprimer les différentes colonnes détaillant le retard
On ne s'intéresse qu'au retard total, on peut donc supprimer les colonnes détaillant les raisons du retard

# Modifier une colonne
Les colonnes DepTime, CRSDepTime, ArrTime et CRSArrTime sont formatées de la façon suivant : hhmm. Par exemple, un DepTime de 724 correspond à 7h24 le matin. On souhaite modifier cette colonne pour avoir le temps en minutes. Par exemple, 7h24 devient 444.

### On commence par modifier la colonne DepTime

In [ ]:
df['DepTime'] = (df.DepTime // 100) * 60 + df.DepTime % 100

In [ ]:
df.tail()

### Exercice : Faire la même transformation pour CRSDepTime, ArrTime et CRSArrTime

### Cas du AirTime

In [ ]:
df.describe()

### A votre avis, que se passe-t-il et comment corriger cela ?

# Synthèse des retards par compagnie
La méthode groupby va nous permettre d'extraire facilement des informations par compagnie : https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.groupby.html

### Création d'un nouveau DataFrame

In [ ]:
df_comp = pd.DataFrame(index=df.UniqueCarrier.unique())

In [ ]:
df_comp

In [ ]:
df_comp.sort_index(inplace=True)

In [ ]:
df_comp.head()

### Première colonne du DataFrame

In [ ]:
df_comp['count'] = df.loc[:, ['UniqueCarrier', 'TotalDelay']].groupby('UniqueCarrier').count()

In [ ]:
df_comp.head()

### Exercice : Continuer à remplir le DataFrame avec la moyenne, l'écart-type, le minimum, le quantile 25%, le quantile 50%, le quantile 75% et le maximum

# Opérations plus complexes dans un groupby

### Application de la fonction finale à plusieurs colonnes

In [ ]:
df.loc[:, ['UniqueCarrier', 'TotalDelay', 'CRSElapsedTime']].groupby('UniqueCarrier').max()

### grouper selon plusieurs colonnes

In [ ]:
df.loc[:, ['Origin', 'Dest', 'TotalDelay']].groupby(['Origin', 'Dest']).max()

### Utilisation d'une lambda fonction

In [ ]:
df.loc[:, ['UniqueCarrier', 'TotalDelay', 'CRSElapsedTime']].groupby('UniqueCarrier').apply(lambda x: 100*(x.TotalDelay / x.CRSElapsedTime).max())

# Merge
On souhaite ajouter les informations df df_comp dans notre DataFrame de départ

In [ ]:
df_full = pd.merge(df, df_comp, how='left', left_on='UniqueCarrier', right_index=True)

In [ ]:
df_full

# Bonus : utilisation avec scikit-learn

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
scaler = MinMaxScaler()

In [ ]:
scaler.fit_transform(df[['DepTime', 'ArrTime']])

### Exercice : appliquez StandardScaler plutôt que MinMaxScaler

# Sauvegarde des données
la méthode to_csv permet d'écrire le DataFrame modifié au format csv. L'argument index=True précise que l'on souhaite écrire l'index dans le fichier

In [ ]:
df.to_csv('datas_cleaned.csv', index=True)